In [1]:
install.packages("remotes")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [2]:
library(remotes)
install_version("vctrs", version = "0.5.0", repos = "http://cran.us.r-project.org")
install_version("stringr", version = "1.4.0", repos = "http://cran.us.r-project.org")

In [3]:
library(stringr)

In [5]:
install.packages("anndata")
install.packages("Signac")

also installing the dependencies ‘rprojroot’, ‘RcppTOML’, ‘here’, ‘png’, ‘reticulate’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Warning message:
“dependencies ‘GenomeInfoDb’, ‘GenomicRanges’, ‘IRanges’, ‘Rsamtools’, ‘S4Vectors’, ‘BiocGenerics’ are not available”
also installing the dependencies ‘dotCall64’, ‘sp’, ‘spam’, ‘SeuratObject’, ‘irlba’, ‘pbapply’, ‘patchwork’, ‘fastmatch’, ‘RcppRoll’


Warning message in install.packages("Signac"):
“installation of package ‘Signac’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [14]:
setRepositories(ind=1:3) # needed to automatically install Bioconductor dependencies
install.packages("Signac")

also installing the dependencies ‘lambda.r’, ‘futile.options’, ‘futile.logger’, ‘snow’, ‘BH’, ‘RCurl’, ‘GenomeInfoDbData’, ‘XVector’, ‘Biostrings’, ‘zlibbioc’, ‘bitops’, ‘BiocParallel’, ‘Rhtslib’, ‘GenomeInfoDb’, ‘GenomicRanges’, ‘IRanges’, ‘Rsamtools’, ‘S4Vectors’, ‘BiocGenerics’


Warning message in install.packages("Signac"):
“installation of package ‘Signac’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [15]:
library(anndata)
library(Signac)
library(Seurat)
library(tidyverse)

data_dir <- "../.."

ERROR: Error in library(Signac): there is no package called ‘Signac’


In [4]:
seurat <- readRDS(path.cat(data_dir, "E14-E15_adata_filtered_seurat_merged_peaks.rds"))

In [5]:
new_annotations <- read.csv(path.cat(data_dir, "endocrine_refinement.csv"))

In [6]:
adapt_obs_names <- function(x){
    tmp <- str_split(x, "-")[[1]]
    if(tmp[3] == "0"){
        prefix <- "E14-5"
        }
    else{
        prefix <- "E15-5"
        }
    return(paste(prefix, "_", tmp[1], "-", tmp[2], sep = ""))
    }

In [7]:
new_annotations$obs_name_adapted <- sapply(new_annotations$X, adapt_obs_names)

In [8]:
seurat <- seurat[,new_annotations$obs_name_adapted]

In [9]:
seurat

An object of class Seurat 
475329 features across 16918 samples within 3 assays 
Active assay: ATAC_merged (228259 features, 228259 variable features)
 2 other assays present: RNA, ATAC
 1 dimensional reduction calculated: lsi

In [10]:
seurat@meta.data$rowname <- rownames(seurat@meta.data)

In [11]:
merged_obs <- merge(seurat@meta.data, new_annotations, by.x="rowname", by.y="obs_name_adapted")

In [12]:
seurat@meta.data <- merged_obs

In [13]:
Idents(seurat) <- seurat@meta.data$refinement

In [14]:
Idents(seurat) <- seurat@meta.data$refinement

In [15]:
celltypes_to_keep = c('Ngn3 low', 'Ngn3 high cycling', 'Ngn3 high,0', 'Ngn3 high,1', 'Eps. progenitors', 'Fev+', 'Fev+ Alpha', 
                   'Fev+ Beta', 'Fev+ Delta,0', 'Fev+ Delta,1', 'Alpha', 'Beta', 'Delta', 'Epsilon')

In [16]:
seurat_filtered <- subset(x=seurat, idents= celltypes_to_keep)


In [17]:
seurat_filtered@meta.data <- seurat@meta.data[seurat@meta.data$refinement %in% celltypes_to_keep,]

In [19]:
saveRDS(seurat_filtered, file=path.cat(data_dir, "seurat_new_annotations.rds")